In [3]:
import numpy as np
import matplotlib.pyplot as plt
import torch 

## 1. 快速了解一个类和包

In [57]:
# python 内置方法访问package结构及其用法
dir(np.linalg) # 访问package结构
help(np.linalg.svd) # 访问具体用法
dir(torch.cuda.is_available())

Help on _ArrayFunctionDispatcher in module numpy.linalg:

svd(a, full_matrices=True, compute_uv=True, hermitian=False)
    Singular Value Decomposition.
    
    When `a` is a 2D array, and ``full_matrices=False``, then it is
    factorized as ``u @ np.diag(s) @ vh = (u * s) @ vh``, where
    `u` and the Hermitian transpose of `vh` are 2D arrays with
    orthonormal columns and `s` is a 1D array of `a`'s singular
    values. When `a` is higher-dimensional, SVD is applied in
    stacked mode as explained below.
    
    Parameters
    ----------
    a : (..., M, N) array_like
        A real or complex array with ``a.ndim >= 2``.
    full_matrices : bool, optional
        If True (default), `u` and `vh` have the shapes ``(..., M, M)`` and
        ``(..., N, N)``, respectively.  Otherwise, the shapes are
        ``(..., M, K)`` and ``(..., K, N)``, respectively, where
        ``K = min(M, N)``.
    compute_uv : bool, optional
        Whether or not to compute `u` and `vh` in addition to `

['__abs__',
 '__add__',
 '__and__',
 '__bool__',
 '__ceil__',
 '__class__',
 '__delattr__',
 '__dir__',
 '__divmod__',
 '__doc__',
 '__eq__',
 '__float__',
 '__floor__',
 '__floordiv__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getnewargs__',
 '__gt__',
 '__hash__',
 '__index__',
 '__init__',
 '__init_subclass__',
 '__int__',
 '__invert__',
 '__le__',
 '__lshift__',
 '__lt__',
 '__mod__',
 '__mul__',
 '__ne__',
 '__neg__',
 '__new__',
 '__or__',
 '__pos__',
 '__pow__',
 '__radd__',
 '__rand__',
 '__rdivmod__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__rfloordiv__',
 '__rlshift__',
 '__rmod__',
 '__rmul__',
 '__ror__',
 '__round__',
 '__rpow__',
 '__rrshift__',
 '__rshift__',
 '__rsub__',
 '__rtruediv__',
 '__rxor__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__sub__',
 '__subclasshook__',
 '__truediv__',
 '__trunc__',
 '__xor__',
 'as_integer_ratio',
 'bit_count',
 'bit_length',
 'conjugate',
 'denominator',
 'from_bytes',
 'imag',
 'numerator',
 'real',
 'to_bytes

In [58]:
plt?? # ipython魔法命令访问更清晰的用法

SyntaxError: invalid syntax (365733421.py, line 1)

## 2. 数据加载

In [4]:
from torch.utils.data import Dataset
from PIL import Image
import os

learning 领域，数据从原始数据load进来的流程： 

`原始数据`(有垃圾数据，有用数据) -> `Dataset预处理数据`(存储有用数据及其label) -> `Dataloader适应训练格式的数据`(以适应训练和访问和访问的形式存储，如batch) -> `nn网络`(训练)


### 2.1 Dataset类

In [1]:
'''
    从Dataset里继承创建适用于supervised learning的数据预处理类
''' 

class MyData_folder(Dataset):
    '''
    ├── train
    │   ├── ants
    │   └── bees
    └── val
        ├── ants
        └── bees
    '''
    # 读取 data_path, label_path
    def __init__(self, root_dir, label):
        self.root_dir = root_dir
        self.label = label
        self.img_dir = os.path.join(self.root_dir, self.label)
        self.img_name_list = os.listdir(self.img_dir)

    # 返回 data, label
    def __getitem__(self, idx):
        img_name = self.img_name_list[idx]
        img_path = os.path.join(self.root_dir, self.label, img_name)
        img_PIL = Image.open(img_path)
        label = self.label
        return img_PIL, label

    # 返回 一类数据的大小
    def __len__(self):
        return len(self.img_name_list)

# 
class MyData_label(Dataset):
    '''
    └── ├── train
    │   ├── ants_image
    │   ├── ants_label
    │   ├── bees_image
    │   └── bees_label
    └── val
        ├── ants_image
        ├── ants_label
        ├── bees_image
        └── bees_label
    '''

    def __init__(self, img_dir, label_dir):
        self.img_dir = img_dir
        self.label_dir = label_dir
        self.img_name_list = os.listdir(self.img_dir)
        self.label_file_list= os.listdir(self.label_dir)

    def __getitem__(self, idx):
        img_name = self.img_name_list[idx]
        img_path = os.path.join(self.img_dir, img_name)
        img_data = Image.open(img_path)

        label_file_name = self.label_file_list[idx]
        label_file_path = os.path.join(self.label_dir, label_file_name)
        with open(label_file_path, 'r') as f:
            label = f.read()

        return img_data, label

    def __len__(self):
        return len(self.img_name_list)

NameError: name 'Dataset' is not defined

In [35]:
# load data from MyData_folder
root_dir = 'dataset/train/'
ants_label_dir = 'ants'
bees_label_dir = 'bees'

ants_dataset_train = MyData_folder(root_dir, ants_label_dir)
bees_dataset_train = MyData_folder(root_dir, bees_label_dir)

# load data from MyData_label
ants_dir = 'dataset_mod/val/ants_image/'
bees_dir = 'dataset_mod/val/bees_image/'
ants_label_dir = 'dataset_mod/val/ants_label/'
bees_label_dir = 'dataset_mod/val/bees_label/'

ants_dataset_val = MyData_label(ants_dir, ants_label_dir)
bees_dataset_val = MyData_label(bees_dir, bees_label_dir)

# 如何使用新设的Dataset类
img_train, label = ants_dataset_train[1]
img_train.show()

img_val, label = ants_dataset_val[1]
img_val.show()

# 对数据进行拼接（多用于数据不足的情况）- 继承Dataset的原始能力
train_dataset = ants_dataset_train + bees_dataset_train
print(f'The num of total train data: {len(train_dataset)}')
val_dataset = ants_dataset_val + bees_dataset_val
print(f'The num of total val data: {len(val_dataset)}')

The num of total train data: 245
The num of total val data: 153


### 2.2 处理数据存放和路径

简单路径处理：

In [20]:
dir_path = r'/home/haoran/pytorch/dataset/train/ants'
img_path_list = os.listdir(dir_path)
img_path_list[0]

root_dir = 'dataset/train'
label_dir = 'ants'
path = os.path.join(root_dir, label_dir)

将文件格式从folder改为但张图片label:

In [29]:
def rename_img_num(img_path: str)-> bool:
    img_name_list = os.listdir(img_path)
    try:
        for i, img_name in enumerate(img_name_list):
            # get old path of imgs
            img_old_path = os.path.join(img_path, img_name)
            
            # get new path of imgs
            new_img_name = f'{i}{os.path.splitext(img_name)[1]}'
            img_new_path = os.path.join(img_path, new_img_name)
    
            # rename old path with new path
            os.rename(img_old_path, img_new_path)
        print('Rename successfully')
    except: 
        print('Rename failed')

def create_label_dir(img_path: str, des_name: str, label:str)-> bool:
    # make a new label folder
    parent_dir = os.path.dirname(img_path)
    des_path = os.path.join(parent_dir, des_name)
    if not os.path.exists(des_path):
        os.makedirs(des_path)
        
    # create label file '.txt'
    num_of_label = len(os.listdir(img_path))
    for i in range(num_of_label):
        file_name = f"{i}.txt"  
        file_path = os.path.join(des_path, file_name)
        with open(file_path, 'w') as file:
            file.write(label)
            
    print('Add label successfully')

In [147]:
img_path = 'dataset_mod/train/bees_image'
rename_img_num(img_path)

img_path = 'dataset_mod/train/bees_image'
des_namlabele = 'bees_label'
label = 'bees'
create_label_dir(img_path, des_name, label)

Rename successfully


## 3. 训练可视化 - Tensorboard的使用

In [11]:
from torch.utils.tensorboard import SummaryWriter

In [155]:
writer = SummaryWriter('logs')
for i in range(100):
    img_path = f'dataset_mod/train/bees_image/{i}.jpg'
    img_PIL = Image.open(img_path)
    img_array = np.array(img_PIL)
    writer.add_image('test', img_array, i, dataformats='HWC')

# y = 2x
for i in range (100):
    writer.add_scalar('y = 4x', 4*i, i)
writer.close()

# 命令行查看： tensorboard --logdir=logs --port=6006
# 同一次训练，不同rollout存在同一文件夹下不同文件中
# 不同次训练，分属不同1命名的文件夹

## 4. 图像预处理 - Transforms的使用

`torchvision.transforms` 库是 PyTorch 中专门用于图像预处理的模块。它提供了**一系列常用的图像转换操作**，这些操作可以方便地对 `PIL` 图像、`NumPy` 数组或 PyTorch 张量进行处理。`transforms` 库中的操作主要用于在深度学习任务中对数据进行**预处理和增强**，比如图像的缩放、裁剪、归一化等。

In [8]:
from torchvision import transforms
from PIL import Image
import cv2

使用流程：从`transforms`库中取出原子功能 -> 构建具体的工具 -> 将`opencv`或者`PIL`读取的图片先转换为`tensor`再放进工具里 -> 输出

使用技巧： 关注输入和输出, 关注参数

In [26]:
# 取出原子功能 -> 构建具体的工具
img_to_tensor = transforms.ToTensor()
# 读取的图片
img_PIL = Image.open(f'dataset_mod/train/bees_image/1.jpg')
img_cv2 = cv2.imread(f'dataset_mod/train/bees_image/1.jpg')
# ToTensor (将opencv或者PIL读取的图片放进工具里 -> 输出)
img_tensor1 = img_to_tensor(img_PIL)
img_tensor2 =  img_to_tensor(img_cv2)
# tensor包含用于学习的大量参数
writer = SummaryWriter('imgs')
writer.add_image('Tensor_img1', img_tensor1) # tensorboard最好传入tensor数据类型
writer.add_image('Tensor_img2', img_tensor2)

# Normalize 
trans_norm = transforms.Normalize([0.05, 0.5, 0.5], [0.5, 0.5, 0.5]) # first_p: mean, second_p = std
img_norm = trans_norm(img_tensor1)
writer.add_image('Norm_img', img_norm)

# Resize 
trans_resize = transforms.Resize((512, 512)) # transforms.Resize(512)
img_resize = trans_resize(img_tensor1)
writer.add_image('Resize_img', img_resize)

# Compose 
trans_compose = transforms.Compose([img_to_tensor, trans_norm, trans_resize]) # 输入list包含transforms对象
img_compose = trans_compose(img_PIL)
writer.add_image('Compose_img', img_compose)

# RandomCrop
trans_random = transforms.RandomCrop(100)
trans_compose2 = transforms.Compose([trans_random, img_to_tensor]) # 先crop在tensor
for i in range(100):
    img_crop = trans_compose2(img_PIL)
    writer.add_image('RandCrop_img', img_crop, i)

## 5. 标准数据集 - torchvision

In [31]:
import torchvision

# CIFAR10
train_set = torchvision.datasets.CIFAR10(root='./datasets', train=True, download=True) # 默认数据类型为PIL
test_set = torchvision.datasets.CIFAR10(root='./datasets', train=False, download=True)
print(train_set[0])
print(train_set.classes)

img, target = test_set[0]
print(img)
print(target)
print(test_set.classes[target])
img.show()

Files already downloaded and verified
Files already downloaded and verified
(<PIL.Image.Image image mode=RGB size=32x32 at 0x7FE16BD15C60>, 6)
['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
<PIL.Image.Image image mode=RGB size=32x32 at 0x7FE16BDAE8C0>
3
cat


## 6. 训练数据导入前的打包 - Dataloader类

In [24]:
import torchvision
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm

In [31]:
test_data = torchvision.datasets.CIFAR10('./datasets', train = False, transform=torchvision.transforms.ToTensor())
img, target = test_data[0]
print(img.shape)
print(test_data.classes[target])

# 对数据集进行打包
test_loader = DataLoader(dataset=test_data, batch_size=64, shuffle=False, num_workers=0, drop_last=True)
'''
    img0, target0 = dataset[0]
    img1, target1 = dataset[1]
    img2, target2 = dataset[2]
    img3, target3 = dataset[3]
    ...
    ---------------------------
    imgs, targets = dataset[:4]
'''
writer = SummaryWriter('dataloader')
for epoch in range(1, 1+2):
    for step, data in enumerate(tqdm(test_loader, desc=f'epoch {epoch}')):
        imgs, targets = data
        # print(imgs.shape)
        # print(targets)
        writer.add_images(f'test_data_drop_last: {epoch}', imgs, step)
writer.close()

torch.Size([3, 32, 32])
cat


epoch 2: 100%|████████████████████████████████| 156/156 [00:03<00:00, 46.53it/s]


## 7. 神经网络

### 7.1 神经网络的基本骨架 - nn.Module

In [33]:
import torch.nn as nn
import torch.nn.functional as F

class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 20, 5)
        self.conv2 = nn.Conv2d(20, 20, 5)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        return F.relu(self.conv2(x))

### 7.2 神经网络 - 卷积层 

In [39]:
import torch
import torch.nn.functional as F

# The 2D convolution (conv2d) in PyTorch expects a 4D input tensor with the following shape:
'''(N, C_in, H, W)
    N: Batch size (number of input samples in a batch)
    C_in: Number of input channels (e.g., 1 for grayscale images, 3 for RGB images)
    H: Height of the input (number of rows)
    W: Width of the input (number of columns)
'''
input = torch.tensor([[1, 2, 0, 3, 1],
                      [1, 2, 1, 3, 1],
                      [2, 2, 0, 2, 1],
                      [1, 2, 3, 3, 2],
                      [1, 3, 0, 2, 1]]).reshape(1, 1, 5, 5)

kernel = torch.tensor([[1, 2, 1],
                       [0, 1, 0],
                       [2, 1, 0]]).reshape(1, 1, 3, 3)

output = F.conv2d(input, kernel, stride=1)
print(output)

output1 = F.conv2d(input, kernel, stride=2)
print(output1)

output2 = F.conv2d(input, kernel, stride=1, padding=1)
print(output2)

tensor([[[[13, 10, 12],
          [12, 14, 19],
          [13, 13, 10]]]])
tensor([[[[13, 12],
          [13, 10]]]])
tensor([[[[ 2,  6,  5,  8,  8],
          [ 7, 13, 10, 12, 11],
          [ 7, 12, 14, 19, 14],
          [ 8, 13, 13, 10, 11],
          [ 5, 11, 11, 13,  8]]]])
